In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## All pairs shortest path



## All pairs shortest path  (APSP)

Up to now, we've consider the shortest paths from a fixed source node $s$.

Of course, we will often want to know the shortest path from **all** nodes in the graph.

How can we use algorithms we have seen to do this?

Just run Bellman-Ford $|V|$ times, once per vertex.

Since one run of Bellman-Ford has $O(|V| \cdot |E|)$ work, the total work is $O(|V|^2 \cdot |E|)$

Can we do better?

## Graph transformations

It would be great if we could instead just run Dijkstra's algorithm $|V|$ times, resulting in $O(|V|\cdot|E| \log |E|)~~~~$ $<<O(|V|^2 \cdot |E|)$

But, we can't run Dijkstra if the graph has negative edge weights.

<br>

**Key idea:** Let's find a transformation from $G=(V,E)$ to $G'=(V,E')$, where all the edges in $E'$ are non-negative.

We'll keep the nodes and edges the same, but just change the weights somehow...

This transformation must preserve two properties:

1. **shortest paths:** The shortest path connecting $u$ to $v$ in $G$ should be the same in $G'$.
2. **non-negative edge weights:** all edges in $G'$ should have weight > 0.

### preserving shortest paths

Consider some path from $v_0$ to $v_k$ in $G$: $p=\langle v_0, v_1, \ldots v_k \rangle$
- let $w(p)$ be the sum of the edge weights in $p$ in the original graph $G$
- let $\delta(v_0, v_k)$ be the weight of the shortest path from $v_0$ to v_k in the original graph $G$
- let $\hat{w}(p)$ be the sum of the edge weights in $p$ in the transformed graph $G'$
- let $\hat{\delta}(v_0, v_k)$ be the weight of the shortest path from $v_0$ to v_k in the transformed graph $G'$

Then, we want a transformation such that:
$w(p) = \delta(v_0, v_k) \leftrightarrow \hat{w}(p) = \hat{\delta}(v_0, v_k)$
- that is, $p$ is a shortest path from $v_0$ to $v_k$ with weights $w$ if and only if $p$ is also a shortest path with weights $w'$


<br><br>
let $f: V \mapsto \mathbb{R}$ be any function mapping vertices to real numbers.




**claim:** Weight transformations of the following form preserve shortest paths:

$$\hat{w}(u,v) = w(u,v) + f(u) - f(v)$$


<img src="figures/johnson_map.png" width=70%/>

<br>

**proof:** First, show that $\hat{w}(p) = w(p) + f(v_0) - f(v_k)$

$
\begin{align}
\hat{w}(p) & = \sum_{i=1}^k \hat{w}(v_{i-1}, v_i) & \hbox{by definition}\\
& = \sum_{i=1}^k \Big( w(v_{i-1}, v_i) + f(v_{i-1}) - f(v_i) \Big)& \hbox{by definition}\\
& = (w(v_0, v_1) + f(v_0) - f(v_1)) + (w(v_1, v_2) + f(v_1) - f(v_2)) + \ldots + (w(v_{k-1}, v_k) + f(v_{k-1}) - f(v_k))& \hbox{expanding}\\
& = w(p) + f(v_0) - f(v_k) & \hbox{cancelling terms}
\end{align}
$

<br><br><br>
Now, consider comparing the weights of two paths $p_i$ and $p_j$ from $v_0$ to $v_k$. 

$\hat{w}(p_i) = w(p_i) + f(v_0) - f(v_k)$  
$\hat{w}(p_j) = w(p_j) + f(v_0) - f(v_k)$

Because the start and end nodes are the same, we have that  
$w(p_i) < w(p_j) \leftrightarrow \hat{w}(p_i) < \hat{w}(p_j)$

$
\begin{align}
\hat{w}(p_i) &< \hat{w}(p_j)\\
w(p_i) + f(v_0) - f(v_k) &< w(p_j) + f(v_0) - f(v_k)\\
w(p_i) &< w(p_j)
\end{align}
$


### ensuring positive edge weights

Not all transformations $\hat{w}(u,v) = w(u,v) + f(u) - f(v)$ will result in positive edge weights.

<br>

We do know this about shortest paths:

**triangle inequality**: 

$
\begin{align}
\delta(u,v) & \le \delta(u,x) + w(x,v)\\
0 & \le w(x,v) + \delta(u,x) - \delta(u,v)
\end{align}
$

which looks a lot like $w(u,v) + f(u) - f(v)$

So, if we can use shortest path as our function $f(v)$, we can ensure that weights are non-negative.

But, what is the source of this shortest path?

<br>
We can't just pick a random node, as not all other nodes may be reachable from it.

  - e.g., $a$ is not reachable from $b$ in our previous example
  
  
<br><br>

Instead, we'll create a "dummy" source node, and connect it with 0 weight to all nodes. This ensures all nodes get a shortest path less than $\infty$.

<br>

  
<center>
<img src="figures/johnson_path.png" width=80%/>
</center>

The new weights are

$\hat{w}(x,v) =  w(x,v) + \delta(s,x) - \delta(s,v)$

### Putting it all together: Johnson's Algorithm

<br>

First, add source $s$ and edges from it to all other vertices with 0 cost:

<center>
<img src="figures/johnson1.jpg"/>
</center>    

Next, run Bellman-Ford on this graph, resulting in distances $D[v]$, indicating the distance from $s$ to each $v$:

<center>
<img src="figures/johnson2.jpg"/>
</center>    


Next, we update the graph using weights $w'(u,v) = w(u,v) + D[u] - D[v].~~~~$ E.g.

$$
\begin{align}
w'(b,c) & = w(b,c) + D[b] - D[v]\\
& = 3 + (-3) - (-1) \\
& = 1
\end{align}
$$  

Let's call this final, modified graph $G'$.
<br>

Next, we run Dijkstra's algorithm for each vertex in $G'$.
- we can use Dijkstra since we've removed negative edges from the graph
- we can do this in parallel
- Let $\delta_{G'}(u,v)$ be the weight of the shortest path from $u$ to $v$ in $G'$, as computed by Dijkstra.

<br>

Finally, we can compute the $\delta_{G}(u,v)$, the shortest path in the original graph $G$, as follows:

$$ \delta_{G}(u,v) = \delta_{G'}(u,v) - D[u] + D[v]$$

E.g.

$$
\begin{align}
\delta_{G}(a,c) & = \delta_{G'}(a,c) - D[a] + D[c]\\
& = 1 - 0 + (-1)\\
& = 0
\end{align}
$$


In [2]:
# our previous implementations of dijkstra and bellman-ford
from heapq import heappush, heappop 
import math

def dijkstra(graph, source):
    def dijkstra_helper(visited, frontier):
        if len(frontier) == 0:
            return visited
        else:
            # Pick next closest node from heap
            distance, node = heappop(frontier)
            if node in visited:
                # Already visited, so ignore this longer path
                return dijkstra_helper(visited, frontier)
            else:
                # We now know the shortest path from source to node.
                # insert into visited dict.
                visited[node] = distance
                # Visit each neighbor of node and insert into heap.
                # We may add same node more than once, heap
                # will keep shortest distance prioritized.
                for neighbor, weight in graph[node]:
                    heappush(frontier, (distance + weight, neighbor))                
                return dijkstra_helper(visited, frontier)
        
    frontier = []
    heappush(frontier, (0, source))
    visited = dict()  # store the final shortest paths for each node.
    return dijkstra_helper(visited, frontier)



def bellmanford(graph, source):
    def bellmanford_helper(distances, k):        
        if k == len(graph): # negative cycle
            return -math.inf
        else:
            # compute new distances
            new_distances = compute_distances(graph, distances)
            
            # check if distances have converged
            if converged(distances, new_distances):
                return distances
            else:                
                return bellmanford_helper(new_distances, k+1)
        
    # initialize
    distances = dict()
    for v in graph:
        if v == source:
            distances[v] = 0
        else:
            distances[v] = math.inf
    return bellmanford_helper(distances, 0)

def compute_distances(graph, distances):
    new_distances = {}
    for v, in_neighbors in graph.items(): # this loop can be done in parallel
        # compute all possible distances from s->v
        v_distances = [distances[v]] 
        for in_neighbor, weight in in_neighbors:
            v_distances.append(distances[in_neighbor] + weight)
        new_distances[v] = min(v_distances)
    return new_distances

def converged(old_distances, new_distances):
    for k in old_distances:
        if old_distances[k] != new_distances[k]:
            return False
    return True


In [5]:
from collections import defaultdict
from pprint import pprint

def johnson(graph):
    # add "dummy" node that links to all nodes with 0 cost.
    for v, in_neighbors in graph.items():
        in_neighbors.add(('s', 0))
    graph['s'] = {}
    
    # call bellmanford with new graph
    distances = bellmanford(graph, 's')
    print('bellman ford distances')
    print(distances)
    
    # create new graph with adjusted weights.
    # b/c we'll be using Dijkstra, we'll use our original representation
    # of a dict from node to set of out-neighbors.
    new_graph = defaultdict(set)
    for v, in_neighbors in graph.items():
        for in_neighbor, weight in in_neighbors:
            new_graph[in_neighbor].add((v, weight + distances[in_neighbor] - distances[v]))
    print('\nnew graph')
    pprint(dict(new_graph))
    
    # now, run dijkstra on each node in the graph
    print('\nunadjusted shortest paths')
    apsp = {}
    for v in new_graph:
        if v != 's':
            distances_from_v = dijkstra(new_graph, v)
            # update final path scores
            print(v, distances_from_v)            
            for u, dist in distances_from_v.items():
                # adjust distances appropriately.
                distances_from_v[u] = dist - distances[v] + distances[u]
            apsp[v] = distances_from_v
    print('\nadjusted shortest paths')
    pprint(apsp)
    return apsp

graph = {
            'a': {('c', 2)},
            'b': {('a', -3)},
            'c': {('a', 1), ('b', 3), ('d', 3)}, 
            'd': {('b', -1)},
        }
    
result = johnson(graph)

bellman ford distances
{'a': 0, 'b': -3, 'c': -1, 'd': -4, 's': 0}

new graph
{'a': {('b', 0), ('c', 2)},
 'b': {('d', 0), ('c', 1)},
 'c': {('a', 1)},
 'd': {('c', 0)},
 's': {('d', 4), ('c', 1), ('b', 3), ('a', 0)}}

unadjusted shortest paths
c {'c': 0, 'a': 1, 'b': 1, 'd': 1}
a {'a': 0, 'b': 0, 'd': 0, 'c': 0}
b {'b': 0, 'd': 0, 'c': 0, 'a': 1}
d {'d': 0, 'c': 0, 'a': 1, 'b': 1}

adjusted shortest paths
{'a': {'a': 0, 'b': -3, 'c': -1, 'd': -4},
 'b': {'a': 4, 'b': 0, 'c': 2, 'd': -1},
 'c': {'a': 2, 'b': -1, 'c': 0, 'd': -2},
 'd': {'a': 5, 'b': 2, 'c': 3, 'd': 0}}


This is an example of a **reduction**! How?

We reduced the APSP problem with negative edge weights, to the APSP problem with positive edge weights.

- The cost of the reduction was $O(|V| \cdot |E|)$, the time to run Bellman-Ford
- The cost to solve the new version of the problem is $O(|V|\cdot|E| \log |E|)$
- Since the cost of the latter is less than the former, this is an *efficient* reduction.

## Cost of Johnson's Algorithm

1. Add edges from $s$:

```python
    for v, in_neighbors in graph.items():
        in_neighbors.add(('s', 0))
```

$O(|V|)$

<br>

2. Run Bellman-Ford

```python
    distances = bellmanford(graph, 's')
```
$O(|V|\cdot |E|)$
       
    
<br>

3. Adjust weights $\hat{w}(x,v) =  w(x,v) + \delta(s,x) - \delta(s,v)$

```python
    new_graph = defaultdict(set)
    for v, in_neighbors in graph.items():
        for in_neighbor, weight in in_neighbors:
            new_graph[in_neighbor].add((v, weight + distances[in_neighbor] - distances[v]))
```

$O(|E|)$
<br>

4. Run Dijkstra $|V|$ times:

```python
    for v in new_graph:
        if v != 's':
            distances_from_v = dijkstra(new_graph, v)
```

$O(|V|\cdot|E| \log |E|)$

5. Adjust final path weights

```python
           for u, dist in distances_from_v.items():
                distances_from_v[u] = dist - distances[v] + distances[u]
```
$O(|E|)$

<br>

Total work is then:

$O(|V|) ~+~ O(|V|\cdot |E|) ~+~ O(|E|) ~+~ O(|V|\cdot|E| \log |E|) ~+~ O(|E|) ~~~ \in ~~~ O(|V|\cdot|E| \log |E|) $